In [1]:
%matplotlib inline

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np
rcParams['figure.figsize'] = 26, 12
rcParams['lines.linewidth'] = 3
matplotlib.rc('xtick', labelsize=20)     
matplotlib.rc('ytick', labelsize=20)


In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
pd.options.display.float_format = '{0:.2f}'.format

@interact(age_max=(30,90,5),retirement_age=(55,70,1),inflation=(1,10,1),interest_on_savings=(0,10,1),\
          annual_savings=(10*1000,120*1000,10000),cost_2_live_another_year=(12*1000,90*1000,12000))
def get_life_df(age_max=90,age_now=43,retirement_age=59, yr_now="2019",annual_savings=30000,\
        cost_2_live_another_year=24000,inflation=3,interest_on_savings=6):
        inflation = inflation*.01
        interest_on_savings = interest_on_savings*.01
        periods=age_max-age_now+1
        retirement_y = retirement_age - age_now
        idx = pd.date_range(f'{yr_now}-12-31', periods=periods, freq='y')
        ts = pd.Series(range(len(idx)), index=idx)
        df = pd.DataFrame(ts,columns=['year'])
        df['age'] = age_now + df.year 
        df['new_money'] = (df['year'] <  retirement_y) * annual_savings
        df['year_end_balance'] = 0
        df['years_spendings'] = 0
        
        df['years_spendings'] = (  cost_2_live_another_year * (1+inflation)**df.year ) * (df.year >= retirement_y)
        
        for i in range(0, len(df)):
                 
                if i > 0:
                        years_savings = df.iloc[i]['new_money']
                        years_spendings = df.iloc[i]['years_spendings']
                        year_end_balance = df.iloc[i-1]['year_end_balance'] + years_savings - years_spendings
                        interest_earned =  year_end_balance * (interest_on_savings)
                        year_end_balance += interest_earned
                        df.iloc[i,df.columns.get_loc('year_end_balance')] = year_end_balance
        
        plt = df['year_end_balance'].plot()
        plt.grid(True)



interactive(children=(IntSlider(value=90, description='age_max', max=90, min=30, step=5), IntSlider(value=43, …